<a href="https://colab.research.google.com/github/Ruchisingh5/Gen_ai/blob/main/fraud_alert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas chromadb langchain llama-cpp-python sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 

In [3]:
import pandas as pd
import pandas as pd

# Load the data
from google.colab import files
uploaded = files.upload()

Saving fraud_flagged.csv to fraud_flagged.csv


In [4]:
df = pd.read_csv("fraud_flagged.csv")

In [5]:

# Flag potential fraud (assuming cluster 3 = fraud)
df['is_fraud'] = df['cluster'].apply(lambda x: 1 if x == 3 else 0)

# Filter fraudulent records for messaging
fraud_data = df[df['is_fraud'] == 1]


In [7]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 7.2 MB/s eta 0:

In [8]:
import chromadb
from sentence_transformers import SentenceTransformer

# Initialize DB and Embedder
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="fraud_context")
embedder = SentenceTransformer("all-MiniLM-L6-v2")  # lightweight open model

# Store some context examples
contexts = [
    {"id": "1", "doc": "Customer made transactions in two distant locations within 10 minutes."},
    {"id": "2", "doc": "Customer exceeded typical spending threshold for the first time."},
    {"id": "3", "doc": "Transaction frequency spiked abnormally today."}
]

# Embed and add to Chroma
for ctx in contexts:
    embedding = embedder.encode(ctx["doc"]).tolist()
    collection.add(documents=[ctx["doc"]], ids=[ctx["id"]], embeddings=[embedding])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
pip install --upgrade pip setuptools wheel


  Using cached setuptools-78.1.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.7 MB/s eta 0:00:00
Using cached setuptools-78.1.0-py3-none-any.whl (1.3 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [15]:
pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 41.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=6008033 sha256=1162c74ad4674a1e3db3613fe51be2016ce3e462f032e2448db272e76b66f0e5
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python


In [17]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="TheBloke/tigerbot-13B-chat-v5-GGUF",
	filename="tigerbot-13b-chat-v5.Q2_K.gguf",
)


tigerbot-13b-chat-v5.Q2_K.gguf:   0%|          | 0.00/5.05G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 23 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--tigerbot-13B-chat-v5-GGUF/snapshots/1f2c59a03295b6f204f5a2bf2de271cd5a3fb820/./tigerbot-13b-chat-v5.Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tigerresearch_tigerbot-13b-chat-v5
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv

In [18]:
prompt = "You are a fraud detection assistant. Write a polite message to a customer whose account shows suspicious activity."
output = llm(prompt=prompt, stop=["</s>"])
print(output['choices'][0]['text'].strip())

llama_perf_context_print:        load time =   14396.29 ms
llama_perf_context_print: prompt eval time =   14396.09 ms /    23 tokens (  625.92 ms per token,     1.60 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   14397.35 ms /    24 tokens


In [20]:
def generate_fraud_message(txn_id, user_profile):
    # Step 1: Retrieve relevant context
    query = f"Transaction details: {user_profile}"
    query_embedding = embedder.encode(query).tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=1)
    retrieved_context = results['documents'][0][0]

    # Step 2: Construct prompt
    prompt = f"""
    You are a fraud detection assistant. A customer with transaction ID {txn_id}
    may have experienced fraud. Use this context: "{retrieved_context}".
    Write a short, friendly message to inform the customer and advise next steps.
    """

    # Step 3: Generate response
    output = llm(prompt=prompt, stop=["</s>"])
    return output["choices"][0]["text"].strip()


In [21]:
for _, row in fraud_data.iterrows():
    txn_id = row['txn_id']
    profile = f"amount: {row['amount']}, time: {row['txn_hour']}, freq: {row['txn_freq']}"
    message = generate_fraud_message(txn_id, profile)
    print(f"\n--- Fraud Alert for Transaction {txn_id} ---")
    print(message)


llama_perf_context_print:        load time =   14396.29 ms
llama_perf_context_print: prompt eval time =   38869.29 ms /    63 tokens (  616.97 ms per token,     1.62 tokens per second)
llama_perf_context_print:        eval time =   14064.41 ms /    15 runs   (  937.63 ms per token,     1.07 tokens per second)
llama_perf_context_print:       total time =   52945.53 ms /    78 tokens
Llama.generate: 17 prefix-match hit, remaining 46 prompt tokens to eval



--- Fraud Alert for Transaction 6.0 ---
_Hey there, I saw a drop in the frequency of your


llama_perf_context_print:        load time =   14396.29 ms
llama_perf_context_print: prompt eval time =   28055.79 ms /    46 tokens (  609.91 ms per token,     1.64 tokens per second)
llama_perf_context_print:        eval time =   14249.40 ms /    15 runs   (  949.96 ms per token,     1.05 tokens per second)
llama_perf_context_print:       total time =   42316.79 ms /    61 tokens
Llama.generate: 17 prefix-match hit, remaining 47 prompt tokens to eval



--- Fraud Alert for Transaction 9.0 ---
Fraud detection message to customer
Could you have experienced fraud?


llama_perf_context_print:        load time =   14396.29 ms
llama_perf_context_print: prompt eval time =   31239.10 ms /    47 tokens (  664.66 ms per token,     1.50 tokens per second)
llama_perf_context_print:        eval time =   14386.32 ms /    15 runs   (  959.09 ms per token,     1.04 tokens per second)
llama_perf_context_print:       total time =   45637.09 ms /    62 tokens
Llama.generate: 17 prefix-match hit, remaining 47 prompt tokens to eval


...

llama_perf_context_print:        load time =   14396.29 ms
llama_perf_context_print: prompt eval time =   29850.17 ms /    49 tokens (  609.19 ms per token,     1.64 tokens per second)
llama_perf_context_print:        eval time =   13960.35 ms /    15 runs   (  930.69 ms per token,     1.07 tokens per second)
llama_perf_context_print:       total time =   43821.96 ms /    64 tokens
Llama.generate: 18 prefix-match hit, remaining 46 prompt tokens to eval



--- Fraud Alert for Transaction 56.0 ---
Write your message here:

The fraud detection system has identified


llama_perf_context_print:        load time =   14396.29 ms
llama_perf_context_print: prompt eval time =   28769.75 ms /    46 tokens (  625.43 ms per token,     1.60 tokens per second)
llama_perf_context_print:        eval time =   13897.64 ms /    15 runs   (  926.51 ms per token,     1.08 tokens per second)
llama_perf_context_print:       total time =   42678.72 ms /    61 tokens



--- Fraud Alert for Transaction 57.0 ---
Message: Hi,
    I noticed there was a sudden


InternalError: Error getting collection: Database error: error returned from database: (code: 1) no such table: collections

In [16]:
from llama_cpp import Llama

llm = Llama(model_path="./models/ggml-mistral-7b.Q4_0.gguf", n_ctx=2048)


ValueError: Model path does not exist: ./models/ggml-mistral-7b.Q4_0.gguf